In [3]:
import numpy as np
import stim
import sys
import itertools
import copy
from bposd.css import css_code
from beliefmatching import BeliefMatching
import pymatching
sys.path.append('../src/')
from QECCircuits import QECCircuit_OneStage

In [5]:
import joblib
hx_dict_2d_read = joblib.load('../data/hx_dict_2d.pkl')
hz_dict_2d_read = joblib.load('../data/hz_dict_2d.pkl')
hx_dict_3d_read = joblib.load('../data/hx_dict_3d.pkl')
hz_dict_3d_read = joblib.load('../data/hz_dict_3d.pkl')
toric_hx_dict_2d = joblib.load('../data/toric_hx_dict_2d.pkl')
toric_hz_dict_2d = joblib.load('../data/toric_hz_dict_2d.pkl')
toric_hx_dict_3d = joblib.load('../data/toric_hx_dict_3d.pkl')
toric_hz_dict_3d = joblib.load('../data/toric_hz_dict_3d.pkl')

# Set up 3D codes

In [7]:
codes_3D = {}

for d in [3, 5]:
    code_3D = css_code(toric_hx_dict_3d[d], toric_hz_dict_3d[d])
    eval_code = code_3D
    # commented out because this requires py 3.7 (run it on a separate kernel to confirm)
    # num_trials = 200
    # dz = DistanceEst_BPOSD(eval_code.hx, eval_code.lx, num_trials) 
    # dx = DistanceEst_BPOSD(eval_code.hz, eval_code.lz, num_trials)
    # eval_code.D = min(dz, dx)
    eval_code.D = d
    print("3D code toric:", code_3D.code_params)
    codes_3D[f'd{d}_toric'] = code_3D

    code_3D = css_code(hx_dict_3d_read[d], hz_dict_3d_read[d])
    eval_code = code_3D
    # dz = DistanceEst_BPOSD(eval_code.hx, eval_code.lx, num_trials)
    # dx = DistanceEst_BPOSD(eval_code.hz, eval_code.lz, num_trials)
    # eval_code.D = min(dz, dx)
    eval_code.D = d
    print("3D code BLP:", code_3D.code_params)
    codes_3D[f'd{d}_blp'] = code_3D


3D code toric: (nan,nan)-[[81,3,3]]
3D code BLP: (nan,nan)-[[27,3,3]]
3D code toric: (nan,nan)-[[375,3,5]]
3D code BLP: (nan,nan)-[[81,3,5]]


In [10]:
for code in codes_3D.values():
    code.hx = code.hx.toarray()
    code.hz = code.hz.toarray()
    code.lx = code.lx.toarray()
    code.lz = code.lz.toarray()

# Memory experiment with 3D codes (toric and BLP)

In [13]:
physical_error_rates = np.logspace(-3, -2, num=6)
circuit_error_params = {"p_i": 0, "p_state_p": 1, "p_m": 1, "p_CX":1, "p_idling_gate": 0}

unnormalized_LFRs_dict = {code_id: [] for code_id in codes_3D.keys()}
normalized_LFRs_dict = {code_id: [] for code_id in codes_3D.keys()} # remember to normalize by kd

num_samples_dict = {'d3_toric': [2000]*6,
                    'd5_toric': [2000]*6,
                    'd3_blp': [2000]*6,
                    'd5_blp': [2000]*6}

In [ ]:
num_shots_per_batch = 10_000
for code_id in ['d3_blp', 'd5_blp', 'd3_toric', 'd5_toric']:
    eval_code = codes_3D[code_id]
    num_rep = eval_code.D
    print("--------------------------------")
    print(f"CODE: {code_id} {eval_code.code_params}")
    for i, p in enumerate(physical_error_rates):
        num_shots = num_samples_dict[code_id][i]
        circuit = QECCircuit_OneStage(eval_code, num_rep, circuit_error_params, p, merged_scheduling=None)
        dem = circuit.detector_error_model(decompose_errors=True)
        matcher = pymatching.Matching.from_detector_error_model(dem)
        sampler = circuit.compile_detector_sampler()
        det_chunks, log_chunks, pred_chunks = [], [], []
        num_batches, leftover = num_shots//num_shots_per_batch, num_shots%num_shots_per_batch
        total_mistakes = 0
        for batch_i in range(num_batches+1):
            if batch_i == num_batches:
                if leftover == 0:
                    break
                batch_detector_vals, batch_logical_vals = sampler.sample(leftover, separate_observables=True)
            else:
                batch_detector_vals, batch_logical_vals = sampler.sample(num_shots_per_batch, separate_observables=True)
            batch_pred_vals = matcher.decode_batch(batch_detector_vals)
            batch_num_mistakes = np.sum(np.any(batch_pred_vals != batch_logical_vals, axis=1))
            
            det_chunks.append(batch_detector_vals)
            log_chunks.append(batch_logical_vals)
            pred_chunks.append(batch_pred_vals)
            total_mistakes += batch_num_mistakes
        detector_vals = np.concatenate(det_chunks, axis=0)
        logical_vals  = np.concatenate(log_chunks,  axis=0)
        if len(detector_vals) != num_shots or len(logical_vals) != num_shots:
            print("incorrect number of shots")
            print(f"det len {len(detector_vals)} | log len {len(logical_vals)}")       
        LFR = total_mistakes/num_shots
        normalized_LFR = 1-(1-LFR)**(1/(eval_code.K * eval_code.D))
        print(f"p={p:.10f} | num shots={num_shots} | LFR={LFR:5f} | normalized LFR={normalized_LFR}")
        unnormalized_LFRs_dict[code_id].append(LFR)
        normalized_LFRs_dict[code_id].append(normalized_LFR)

--------------------------------
CODE: d3_blp (nan,nan)-[[27,3,3]]
p=0.0010000000 | num shots=2000 | LFR=0.013500 | normalized LFR=0.001509077253793789
p=0.0015848932 | num shots=2000 | LFR=0.023500 | normalized LFR=0.0026387932110290002
p=0.0025118864 | num shots=2000 | LFR=0.057000 | normalized LFR=0.0064997840170825105
p=0.0039810717 | num shots=2000 | LFR=0.140000 | normalized LFR=0.01661846301918468
p=0.0063095734 | num shots=2000 | LFR=0.254500 | normalized LFR=0.03210662679430243
p=0.0100000000 | num shots=2000 | LFR=0.502500 | normalized LFR=0.07464080921962202
--------------------------------
CODE: d5_blp (nan,nan)-[[81,3,5]]
p=0.0010000000 | num shots=2000 | LFR=0.003000 | normalized LFR=0.00020028054252707594
p=0.0015848932 | num shots=2000 | LFR=0.017000 | normalized LFR=0.0011424241917163958
p=0.0025118864 | num shots=2000 | LFR=0.064000 | normalized LFR=0.004399613386810053
p=0.0039810717 | num shots=2000 | LFR=0.201500 | normalized LFR=0.014889394324754557
p=0.0063095734

In [ ]:
# with idling, compact
import matplotlib.pyplot as plt

printed_code_params = {code_id: f"{code.code_params}".split('-')[1] for code_id, code in codes_3D.items()}

plt.rcParams.update({'font.size': 18})
fig = plt.figure()
fig.set_figwidth(14.4)
fig.set_figheight(10)

plt.plot(physical_error_rates, normalized_LFRs_dict['d3_toric'], marker='o', linestyle=':', color='C0', label=f"toric {printed_code_params['d3_toric']}", linewidth=3)
plt.plot(physical_error_rates, normalized_LFRs_dict['d5_toric'], marker='o', linestyle=':', color='C1', label=f"toric {printed_code_params['d5_toric']}", linewidth=3)
plt.plot(physical_error_rates, normalized_LFRs_dict['d3_blp'], marker='o', linestyle='-', color='C0', label=f"blp {printed_code_params['d3_blp']}", linewidth=3)
plt.plot(physical_error_rates, normalized_LFRs_dict['d5_blp'], marker='o', linestyle='-', color='C1', label=f"blp {printed_code_params['d5_blp']}", linewidth=3)
# plt.plot(physical_error_rates, normalized_LFRs_dict['d3_pentagon'], marker='o', linestyle='--', color='C0', label=f"pentagon {printed_code_params['d3_pentagon']}", linewidth=3)

plt.xscale('log')
plt.yscale('log')
# plt.ylim(10e-6, 2*10e-2)

# Labels and title
plt.xlabel('Physical Error Rate')
plt.ylabel('Normalized Logical Failure Rate')
plt.title('Normalized Logical Failure Rate vs Physical Error Rate\n for Memory Experiment with 3D Codes')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()

plt.tight_layout()
plt.show()